Deep Learning
=============

Digit Recogniser using Convolution Neural Network
------------

Previously in `fullyconnected.ipynb` and `regularization.ipynb`, we  have trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

 In this project, we enhance on the earlier work and try to use state of art Convolution Neural Netowrk to get even better accuracy with digit recognition.
To know more about the theoritical foundation of Convolution Neural network, please click here [Understanding Convolution Neural Networks](https://deepdatascience.wordpress.com/2017/02/07/convolutional-networks/)

In [19]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import datetime
import random

In [20]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)
print(datetime.datetime.now())

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)
2017-03-28 21:17:06.539158


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [21]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)
print( datetime.datetime.now() )

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)
2017-03-28 21:17:09.129803


In [22]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])
  # gives [True, False, True, True,...] based on whethe predictions and lables is equal or not
  #correct_prediction = tf.equal( tf.argmax(predictions, 1), tf.argmax(labels, 1)  )
  # cast converts bool to float [True, False, True, True] => [1,0,1,1]
  #return   tf.cast( (tf.reduce_mean(tf.cast(correct_prediction, tf.float32))), float)
print(datetime.datetime.now())

2017-03-28 21:17:13.350547


Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.


In [23]:
def GetOptimiser(tf, gradient_optimizer, loss, rate):
    #print('gradient_optimizer is :: ',gradient_optimizer)
    if gradient_optimizer == 'GradientDescent':
        print('SELECTED OPTIMISER :: Gradient Descent optimisier with default rate - 0.05')
        return tf.train.GradientDescentOptimizer(0.05).minimize(loss)        
    elif  gradient_optimizer == 'Adam':
        print('SELECTED OPTIMISER :: Adam with learning rate - ', rate)
        return tf.train.AdamOptimizer(rate).minimize(loss)
    else:
        print('SELECTED OPTIMISER :: Invalid optimiser selected. Valid Optimiser : {GradientDescent | Adam}')
        return False

def InitNRunTfModel(num_steps,  batch_size, patch_size, depth, num_hidden,\
                report_interval_steps,  random_seed, gradient_optimizer, learning_rate):
    print('patch size ', patch_size)
    graph = tf.Graph()
    with graph.as_default():
      # Input data.
      tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)
              # Variables.
      layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, num_channels, depth], stddev=0.1))
      layer1_biases = tf.Variable(tf.zeros([depth]))
      layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, depth, depth], stddev=0.1))
      layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
      layer3_weights = tf.Variable(tf.truncated_normal(
          [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
      layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
      layer4_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_labels], stddev=0.1))
      layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
               # Model.
      def model(data):       
            print('Using  Vanilla With no max pool. Drop out not supported yet')
            strides = [1, 2, 2, 1]   # A stride of sliding windows for each dimension. Here two dimension hence 4 stride param     
            conv = tf.nn.conv2d(data, layer1_weights, strides, padding='SAME') # sweep a 2-D filter over a batch of images, with tf.nn.conv2d  
            hidden = tf.nn.relu(conv + layer1_biases)
            conv = tf.nn.conv2d(hidden, layer2_weights, strides, padding='SAME')
            hidden = tf.nn.relu(conv + layer2_biases)
            shape = hidden.get_shape().as_list()
            reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
            hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
            return tf.matmul(hidden, layer4_weights) + layer4_biases        
              # Training computation.
      logits = model(tf_train_dataset)
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))      
                # Optimizer.
      optimizer = GetOptimiser(tf, gradient_optimizer, loss, rate = learning_rate)
      #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
      #optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)
              # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax( model(tf_valid_dataset))
      test_prediction = tf.nn.softmax( model(tf_test_dataset))
    #print('TensorFlow Graph Initialisation completed at ',datetime.datetime.now())
    
#def RunTfModel(graph, num_steps, batch_size, report_interval_steps = 50, random_seed = 12  ):        
    ################
    ### Run TF model
    ###############
    start_time = datetime.datetime.now()
    with tf.Session(graph=graph) as session:
      #tf.global_variables_initializer().run()
      tf.initialize_all_variables().run()
      print('Initialized')
      for step in range(num_steps):
        random.seed(random_seed*step)    
        offset = random.randint( 1, train_labels.shape[0] - batch_size )        
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)        
        if (step % report_interval_steps == 0):
            print('Step %4d :: Minibatch ==> Loss: %f .Accuracy: %.1f%% .Validation Acc.: %.1f%% DataOffset:%7d'\
                    % (step, l, accuracy(predictions, batch_labels), accuracy(valid_prediction.eval(), valid_labels), offset ))          
      print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))     
      #tf.reset_default_graph()
      session.close()    
    tf.reset_default_graph()    
    print('Total Time taken', datetime.datetime.now() - start_time)
    
def RunNN(num_steps = 101, batch_size = 16, patch_size = 5, depth = 16, num_hidden = 64 \
         , report_interval_steps = 50, random_seed = 12, gradient_optimizer = 'None'\
         , learning_rate = 0.0001):        
                #def RunNN(num_steps , batch_size , patch_size , depth , num_hidden, report_interval_steps , random_seed ):            
    InitNRunTfModel( num_steps = num_steps, batch_size = batch_size, patch_size = patch_size, \
            depth = depth, num_hidden = num_hidden , report_interval_steps = report_interval_steps,\
            random_seed = random_seed, gradient_optimizer= gradient_optimizer, learning_rate = learning_rate)
    #RunTfModel(graph, num_steps = num_steps , batch_size = batch_size ,report_interval_steps = report_interval_steps, random_seed = random_seed  )
    
print('NN Function Init Completed', datetime.datetime.now()) 

NN Function Init Completed 2017-03-28 21:17:20.650543


Patch Size :  7 x 7 
-------------------------------------

In [28]:
RunNN(num_steps = 5001, batch_size = 100, patch_size = 7, depth = 16, num_hidden = 64\
     , report_interval_steps =1000, gradient_optimizer = 'GradientDescent' )

patch size  7
Using  Vanilla With no max pool. Drop out not supported yet
SELECTED OPTIMISER :: Gradient Descent optimisier with default rate - 0.05
Using  Vanilla With no max pool. Drop out not supported yet
Using  Vanilla With no max pool. Drop out not supported yet
Initialized
Step    0 :: Minibatch ==> Loss: 2.896104 .Accuracy: 6.0% .Validation Acc.: 13.8% DataOffset: 168800
Step 1000 :: Minibatch ==> Loss: 0.650454 .Accuracy: 80.0% .Validation Acc.: 84.8% DataOffset:   7689
Step 2000 :: Minibatch ==> Loss: 0.377741 .Accuracy: 87.0% .Validation Acc.: 87.0% DataOffset:  99859
Step 3000 :: Minibatch ==> Loss: 0.385118 .Accuracy: 87.0% .Validation Acc.: 87.7% DataOffset: 183851
Step 4000 :: Minibatch ==> Loss: 0.312888 .Accuracy: 91.0% .Validation Acc.: 88.4% DataOffset:  36536
Step 5000 :: Minibatch ==> Loss: 0.456389 .Accuracy: 85.0% .Validation Acc.: 88.8% DataOffset: 129488
Test accuracy: 94.3%
Total Time taken 0:05:48.733186


Patch Size :  5 x 5 : Best
-------------------------------------

In [26]:
RunNN(num_steps = 5001, batch_size = 100, patch_size = 5, depth = 16, num_hidden = 64\
     , report_interval_steps =1000, gradient_optimizer = 'GradientDescent' )

patch size  5
Using  Vanilla With no max pool. Drop out not supported yet
SELECTED OPTIMISER :: Gradient Descent optimisier with default rate - 0.05
Using  Vanilla With no max pool. Drop out not supported yet
Using  Vanilla With no max pool. Drop out not supported yet
Initialized
Step    0 :: Minibatch ==> Loss: 3.365601 .Accuracy: 14.0% .Validation Acc.: 9.9% DataOffset: 168800
Step 1000 :: Minibatch ==> Loss: 0.607137 .Accuracy: 82.0% .Validation Acc.: 84.1% DataOffset:   7689
Step 2000 :: Minibatch ==> Loss: 0.362161 .Accuracy: 87.0% .Validation Acc.: 85.9% DataOffset:  99859
Step 3000 :: Minibatch ==> Loss: 0.415511 .Accuracy: 86.0% .Validation Acc.: 86.9% DataOffset: 183851
Step 4000 :: Minibatch ==> Loss: 0.372233 .Accuracy: 90.0% .Validation Acc.: 87.7% DataOffset:  36536
Step 5000 :: Minibatch ==> Loss: 0.427996 .Accuracy: 90.0% .Validation Acc.: 87.9% DataOffset: 129488
Test accuracy: 94.2%
Total Time taken 0:03:41.117379


In [30]:
RunNN(num_steps = 5001, batch_size = 16, patch_size = 5, depth = 16, num_hidden = 64\
     , report_interval_steps =1000, gradient_optimizer = 'GradientDescent' )

patch size  5
Using  Vanilla With no max pool. Drop out not supported yet
SELECTED OPTIMISER :: Gradient Descent optimisier with default rate - 0.05
Using  Vanilla With no max pool. Drop out not supported yet
Using  Vanilla With no max pool. Drop out not supported yet
Initialized
Step    0 :: Minibatch ==> Loss: 3.549231 .Accuracy: 12.5% .Validation Acc.: 10.0% DataOffset: 168871
Step 1000 :: Minibatch ==> Loss: 0.294258 .Accuracy: 93.8% .Validation Acc.: 82.4% DataOffset:   7693
Step 2000 :: Minibatch ==> Loss: 0.525900 .Accuracy: 81.2% .Validation Acc.: 84.6% DataOffset:  99901
Step 3000 :: Minibatch ==> Loss: 0.569500 .Accuracy: 81.2% .Validation Acc.: 85.8% DataOffset: 183928
Step 4000 :: Minibatch ==> Loss: 0.878385 .Accuracy: 75.0% .Validation Acc.: 86.3% DataOffset:  36552
Step 5000 :: Minibatch ==> Loss: 0.354826 .Accuracy: 87.5% .Validation Acc.: 87.0% DataOffset: 129543
Test accuracy: 93.6%
Total Time taken 0:00:54.464790


Smaller Patch Size :  3 x 3 
-------------------------------------

In [24]:
RunNN(num_steps = 5001, batch_size = 100, patch_size = 3, depth = 16, num_hidden = 64\
     , report_interval_steps =1000, gradient_optimizer = 'GradientDescent' )

patch size  3
Using  Vanilla With no max pool. Drop out not supported yet
SELECTED OPTIMISER :: Gradient Descent optimisier with default rate - 0.05
Using  Vanilla With no max pool. Drop out not supported yet
Using  Vanilla With no max pool. Drop out not supported yet
Initialized
Step    0 :: Minibatch ==> Loss: 3.445589 .Accuracy: 7.0% .Validation Acc.: 9.0% DataOffset: 168800
Step 1000 :: Minibatch ==> Loss: 0.616753 .Accuracy: 83.0% .Validation Acc.: 81.3% DataOffset:   7689
Step 2000 :: Minibatch ==> Loss: 0.424078 .Accuracy: 88.0% .Validation Acc.: 84.0% DataOffset:  99859
Step 3000 :: Minibatch ==> Loss: 0.461060 .Accuracy: 86.0% .Validation Acc.: 84.7% DataOffset: 183851
Step 4000 :: Minibatch ==> Loss: 0.364323 .Accuracy: 86.0% .Validation Acc.: 85.6% DataOffset:  36536
Step 5000 :: Minibatch ==> Loss: 0.547264 .Accuracy: 82.0% .Validation Acc.: 85.9% DataOffset: 129488
Test accuracy: 92.4%
Total Time taken 0:02:27.722717


Smallest Patch Size :  1 x 1 
-------------------------------------

In [25]:
RunNN(num_steps = 5001, batch_size = 100, patch_size = 1, depth = 16, num_hidden = 64\
     , report_interval_steps =1000 , gradient_optimizer = 'GradientDescent' )

patch size  1
Using  Vanilla With no max pool. Drop out not supported yet
SELECTED OPTIMISER :: Gradient Descent optimisier with default rate - 0.05
Using  Vanilla With no max pool. Drop out not supported yet
Using  Vanilla With no max pool. Drop out not supported yet
Initialized
Step    0 :: Minibatch ==> Loss: 4.267948 .Accuracy: 12.0% .Validation Acc.: 10.0% DataOffset: 168800
Step 1000 :: Minibatch ==> Loss: 2.302978 .Accuracy: 8.0% .Validation Acc.: 10.0% DataOffset:   7689
Step 2000 :: Minibatch ==> Loss: 2.301145 .Accuracy: 14.0% .Validation Acc.: 10.0% DataOffset:  99859
Step 3000 :: Minibatch ==> Loss: 2.304610 .Accuracy: 4.0% .Validation Acc.: 10.0% DataOffset: 183851
Step 4000 :: Minibatch ==> Loss: 2.302458 .Accuracy: 11.0% .Validation Acc.: 10.0% DataOffset:  36536
Step 5000 :: Minibatch ==> Loss: 2.303481 .Accuracy: 10.0% .Validation Acc.: 10.0% DataOffset: 129488
Test accuracy: 10.0%
Total Time taken 0:01:00.377971


Patch Size :  5 x 5 + AdamOptimiser :  Best, new conf
-------------------------------------

In [32]:
RunNN( num_steps = 5001, batch_size = 100, patch_size = 5, depth = 16, num_hidden = 64 \
            , report_interval_steps = 1000 , random_seed = 12, gradient_optimizer = 'Adam'\
      , learning_rate = 0.001 )

patch size  5
Using  Vanilla With no max pool. Drop out not supported yet
SELECTED OPTIMISER :: Adam with learning rate -  0.001
Using  Vanilla With no max pool. Drop out not supported yet
Using  Vanilla With no max pool. Drop out not supported yet
Initialized
Step    0 :: Minibatch ==> Loss: 4.727360 .Accuracy: 7.0% .Validation Acc.: 10.0% DataOffset: 168800
Step 1000 :: Minibatch ==> Loss: 0.514220 .Accuracy: 86.0% .Validation Acc.: 85.9% DataOffset:   7689
Step 2000 :: Minibatch ==> Loss: 0.280834 .Accuracy: 91.0% .Validation Acc.: 88.0% DataOffset:  99859
Step 3000 :: Minibatch ==> Loss: 0.332831 .Accuracy: 90.0% .Validation Acc.: 88.8% DataOffset: 183851
Step 4000 :: Minibatch ==> Loss: 0.266403 .Accuracy: 92.0% .Validation Acc.: 89.1% DataOffset:  36536
Step 5000 :: Minibatch ==> Loss: 0.388152 .Accuracy: 89.0% .Validation Acc.: 90.1% DataOffset: 129488
Test accuracy: 95.5%
Total Time taken 0:03:28.837959


Problem 1 : Max-Pool implementation : Min RAM : 4GB
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.



In [37]:
def GetOptimiser(tf, gradient_optimizer, loss, rate):
    #print('gradient_optimizer is :: ',gradient_optimizer)
    if gradient_optimizer == 'GradientDescent':
        print('SELECTED OPTIMISER :: Gradient Descent optimisier with default rate - 0.05')
        return tf.train.GradientDescentOptimizer(0.05).minimize(loss)        
    elif  gradient_optimizer == 'Adam':
        print('SELECTED OPTIMISER :: Adam with learning rate - ', rate)
        return tf.train.AdamOptimizer(rate).minimize(loss)
    else:
        print('SELECTED OPTIMISER :: Invalid optimiser selected. Valid Optimiser : {GradientDescent | Adam}')
        return False

def InitNRunTfModel(num_steps,  batch_size, patch_size, depth, num_hidden,\
                report_interval_steps,  random_seed, gradient_optimizer, learning_rate, use_2x2_maxpool ):
    print('patch size ', patch_size)
    graph = tf.Graph()
    with graph.as_default():
      # Input data.
      tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)
              # Variables.
      layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, num_channels, depth], stddev=0.1))
      layer1_biases = tf.Variable(tf.zeros([depth]))
      layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size, patch_size, depth, depth], stddev=0.1))
      layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
      layer3_weights = tf.Variable(tf.truncated_normal(
          [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
      layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
      layer4_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_labels], stddev=0.1))
      layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
      # We need placeholder because we need to turn dropout on  for training only and  imp. not for other validation and test set/
      dropout_probability_train  = tf.placeholder(tf.float32)
              # Model.
      def model(data, use_2x2_maxpool, dropout_keep_probability = 1, defn = 'Train'):
        if use_2x2_maxpool == 0:
            print('Using  Vanilla With no max pool. Drop out not supported yet')
            strides = [1, 2, 2, 1]   # A stride of sliding windows for each dimension. Here two dimension hence 4 stride param     
            conv = tf.nn.conv2d(data, layer1_weights, strides, padding='SAME') # sweep a 2-D filter over a batch of images, with tf.nn.conv2d  
            hidden = tf.nn.relu(conv + layer1_biases)
            conv = tf.nn.conv2d(hidden, layer2_weights, strides, padding='SAME')
            hidden = tf.nn.relu(conv + layer2_biases)
            shape = hidden.get_shape().as_list()
            reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
            hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
            return tf.matmul(hidden, layer4_weights) + layer4_biases      
        else:
            print('Using  2x2 max pool. Dropout suppported for '+defn+' at keep probability rate', dropout_keep_probability)      
            strides = [1, 1, 1, 1]   # A stride of sliding windows for each dimension. Here two dimension hence 4 stride param     
            #conv1 = tf.nn.conv2d(data, layer1_weights, strides = [1 ,1 ,1 , 1], padding='SAME') # sweep a 2-D filter over a batch of images, with tf.nn.conv2d  
            #hidden1 = tf.nn.relu(tf.nn.conv2d(data, layer1_weights, strides = [1 ,1 ,1 , 1], padding='SAME')\
            #                     + layer1_biases)
            #max_pool1 = tf.nn.max_pool(\
            #                ( tf.nn.relu(tf.nn.conv2d(data, layer1_weights, strides = [1 ,1 ,1 , 1], padding='SAME')\
            #                     + layer1_biases) )\
            #                           , ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME' )
            #conv2 = tf.nn.conv2d(max_pool1, layer2_weights, strides = [1 ,1 ,1, 1] , padding='SAME')
            #hidden2 = tf.nn.relu(tf.nn.conv2d(max_pool1, layer2_weights, strides = [1 ,1 ,1, 1] , padding='SAME')\
            #                     + layer2_biases)
            #max_pool2 = tf.nn.max_pool(\
            #            ( tf.nn.relu(tf.nn.conv2d(max_pool1, layer2_weights, strides = [1 ,1 ,1, 1], padding='SAME')\
            #                     + layer2_biases) )\
            #                           , ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME' )
            ############################ Single Max pool variable
            max_pool2 = tf.nn.max_pool(( \
                   #Hidden Layer 2 
                   tf.nn.relu(\
                     #Conv Layer 2                 
                     tf.nn.conv2d(( \
                         #max pool layer 1
                         tf.nn.max_pool(\
                             #hidden_layer 1
                             ( tf.nn.relu(\
                                  #Conv Layer 1        
                                  tf.nn.conv2d(data, layer1_weights, strides = [1 ,1 ,1 , 1], padding='SAME')\
                                  #Conv Layer 1 end
                              + layer1_biases) )\
                              #hidden_layer 1 end                                        
                         , ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME' )\
                         #max pool layer 1 end
                      ), layer2_weights, strides = [1 ,1 ,1, 1], padding='SAME')\
                      #Conv Layer 2 end
                   + layer2_biases) )\
                   ##Hidden Layer 2  end
              , ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME' )
              # MAx Pool2 end          
            
            shape = max_pool2.get_shape().as_list()
            #reshape = tf.reshape(max_pool2, [shape[0], shape[1] * shape[2] * shape[3]])
            #hidden3 = tf.nn.relu(tf.matmul(\
            #                         ( tf.reshape(max_pool2, [shape[0], shape[1] * shape[2] * shape[3]]) )\
            #                               , layer3_weights) + layer3_biases)
            return tf.matmul(\
                          # drop-out 
                          tf.nn.dropout(\
                             #hidden layer 3
                             ( tf.nn.relu(tf.matmul(\
                                     ( tf.reshape(max_pool2, [shape[0], shape[1] * shape[2] * shape[3]]) )\
                                           , layer3_weights) + layer3_biases) )\
                             # hidden layer 3 end
                           , dropout_keep_probability)\
                           # drop-out end             
                    , layer4_weights) + layer4_biases

              # Training computation.
      logits = model(tf_train_dataset, use_2x2_maxpool, 0.5, 'Training Data')
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))      
                # Optimizer.
      optimizer = GetOptimiser(tf, gradient_optimizer, loss, rate = learning_rate)
      #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
      #optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)
              # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax( model(tf_valid_dataset, use_2x2_maxpool, 1, 'Validation Data' ))
      test_prediction = tf.nn.softmax( model(tf_test_dataset, use_2x2_maxpool, 1, 'Test Data'))
    #print('TensorFlow Graph Initialisation completed at ',datetime.datetime.now())
    
#def RunTfModel(graph, num_steps, batch_size, report_interval_steps = 50, random_seed = 12  ):        
    ################
    ### Run TF model
    ###############
    start_time = datetime.datetime.now()
    with tf.Session(graph=graph) as session:
      #tf.global_variables_initializer().run()
      tf.initialize_all_variables().run()
      print('Initialized')
      for step in range(num_steps):
        random.seed(random_seed*step)    
        offset = random.randint( 1, train_labels.shape[0] - batch_size )        
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)        
        if (step % report_interval_steps == 0):
            print('Step %4d :: Minibatch ==> Loss: %f .Accuracy: %.1f%% .Validation Acc.: %.1f%% DataOffset:%7d'\
                    % (step, l, accuracy(predictions, batch_labels), accuracy(valid_prediction.eval(), valid_labels), offset ))          
      print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))     
      #tf.reset_default_graph()
      session.close()    
    tf.reset_default_graph()    
    print('Total Time taken', datetime.datetime.now() - start_time)
    
def RunNN(num_steps = 101, batch_size = 16, patch_size = 5, depth = 16, num_hidden = 64 \
         , report_interval_steps = 50, random_seed = 12, gradient_optimizer = 'None'\
         , learning_rate = 0.0001, use_2x2_maxpool = 0 ):        
                #def RunNN(num_steps , batch_size , patch_size , depth , num_hidden, report_interval_steps , random_seed ):            
    InitNRunTfModel( num_steps = num_steps, batch_size = batch_size, patch_size = patch_size, \
            depth = depth, num_hidden = num_hidden , report_interval_steps = report_interval_steps,\
            random_seed = random_seed, gradient_optimizer= gradient_optimizer, learning_rate = learning_rate\
            , use_2x2_maxpool = use_2x2_maxpool)
    #RunTfModel(graph, num_steps = num_steps , batch_size = batch_size ,report_interval_steps = report_interval_steps, random_seed = random_seed  )
    
print('NN Function Init Completed', datetime.datetime.now())  


NN Function Init Completed 2017-03-28 22:20:30.701819


Patch : 5 x 5 + MaxPool
----------------------

In [38]:
RunNN( num_steps = 5001, batch_size = 16, patch_size = 5, depth = 16, num_hidden = 64 \
            , report_interval_steps = 1000 , random_seed = 12, gradient_optimizer = 'Adam'\
      , learning_rate = 0.01, use_2x2_maxpool = 1 )

patch size  5
Using  2x2 max pool. Dropout suppported for Train at keep probability rate 0.5
SELECTED OPTIMISER :: Adam with learning rate -  0.01
Using  2x2 max pool. Dropout suppported for Valid at keep probability rate 1
Using  2x2 max pool. Dropout suppported for Test at keep probability rate 1
Initialized
Step    0 :: Minibatch ==> Loss: 4.404894 .Accuracy: 18.8% .Validation Acc.: 10.0% DataOffset: 168871
Step 1000 :: Minibatch ==> Loss: 0.596764 .Accuracy: 81.2% .Validation Acc.: 80.6% DataOffset:   7693
Step 2000 :: Minibatch ==> Loss: 0.552265 .Accuracy: 81.2% .Validation Acc.: 80.8% DataOffset:  99901
Step 3000 :: Minibatch ==> Loss: 2.019364 .Accuracy: 62.5% .Validation Acc.: 73.4% DataOffset: 183928
Step 4000 :: Minibatch ==> Loss: 0.937754 .Accuracy: 68.8% .Validation Acc.: 80.7% DataOffset:  36552
Step 5000 :: Minibatch ==> Loss: 1.022063 .Accuracy: 68.8% .Validation Acc.: 81.7% DataOffset: 129543
Test accuracy: 89.0%
Total Time taken 0:08:10.221837


---
Problem 2 : Drop Out 
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [36]:
RunNN( num_steps = 5001, batch_size = 100, patch_size = 5, depth = 16, num_hidden = 64 \
            , report_interval_steps = 1000 , random_seed = 12, gradient_optimizer = 'Adam'\
      , learning_rate = 0.001, use_2x2_maxpool = 1 )

patch size  5
Using  2x2 max pool. Dropout suppported for train at rate 0.5
SELECTED OPTIMISER :: Adam with learning rate -  0.001
Using  2x2 max pool. Dropout suppported for train at rate 1
Using  2x2 max pool. Dropout suppported for train at rate 1
Initialized
Step    0 :: Minibatch ==> Loss: 5.931297 .Accuracy: 12.0% .Validation Acc.: 10.0% DataOffset: 168800
Step 1000 :: Minibatch ==> Loss: 0.716753 .Accuracy: 77.0% .Validation Acc.: 85.2% DataOffset:   7689
Step 2000 :: Minibatch ==> Loss: 0.470212 .Accuracy: 87.0% .Validation Acc.: 87.2% DataOffset:  99859
Step 3000 :: Minibatch ==> Loss: 0.519879 .Accuracy: 83.0% .Validation Acc.: 87.6% DataOffset: 183851
Step 4000 :: Minibatch ==> Loss: 0.421558 .Accuracy: 85.0% .Validation Acc.: 88.3% DataOffset:  36536
Step 5000 :: Minibatch ==> Loss: 0.564555 .Accuracy: 85.0% .Validation Acc.: 88.7% DataOffset: 129488
Test accuracy: 94.9%
Total Time taken 0:17:33.244888


Misc :
--------------------------
**Understanding strides parameter in Conv2d**
strides = [1, 2, 2, 1]  
- A stride of sliding windows for each dimension. Here two dimension hence 4 stride param   

- strides = [batch, height, width, channel].

with [1, 2 , 2 , 1] we are saying that we dont want to slide across batch and channel and that we want to slide by 2 when it comes to height and width i.e

10 11 12 13 14 ...

20 21 22 23 24 ...

30 31 32 33 34 ...


| Stride  (patch-2 x2)  | [1, 1, 1, 1]  | [1, 2, 2, 1]  |
| ------------- |:-------------:| -----:|
| Filter1       | 10 11         | 10 11 |
|               | 20 21         | 20 21 |
| Filter2       | 11 12         | 12 13 |
|               | 21 22         | 22 23 |            

To understand the code more and better visit, https://bikalblogs.wordpress.com/2017/02/10/tensorflow-convolution-nets-and-params/

